In [65]:
import pandas as pd
import numpy as np


In [66]:
def create_items_dataframe_integer(num_items, value_min, value_max, cost_min, cost_max):
    """
    Create a pandas DataFrame with N items, each having a randomly sampled integer value and cost.
    
    Parameters:
    - num_items (int): Number of items to generate.
    - value_min (int): Minimum possible value for the 'value' column.
    - value_max (int): Maximum possible value for the 'value' column.
    - cost_min (int): Minimum possible cost for the 'cost' column.
    - cost_max (int): Maximum possible cost for the 'cost' column.
    
    Returns:
    - DataFrame: A pandas DataFrame with columns ['item', 'value', 'cost'].
    """
    import pandas as pd
    import numpy as np

    data = {
        'item': [f'Item_{i+1}' for i in range(num_items)],
        'value': np.random.randint(value_min, value_max + 1, num_items),
        'cost': np.random.randint(cost_min, cost_max + 1, num_items),
    }
    return pd.DataFrame(data)


def create_knapsack_capacities_integer(num_knapsacks, capacity_min, capacity_max):
    """
    Create a list of integer capacities for knapsacks, with values randomly sampled within a given range.
    
    Parameters:
    - num_knapsacks (int): Number of knapsacks to generate capacities for.
    - capacity_min (int): Minimum possible capacity for each knapsack.
    - capacity_max (int): Maximum possible capacity for each knapsack.
    
    Returns:
    - List[int]: A list of capacities for the knapsacks.
    """
    import numpy as np
    return list(np.random.randint(capacity_min, capacity_max + 1, num_knapsacks))


# Example usage
df = create_items_dataframe_integer(50, 10, 100, 5, 50)
print(df)

knapsack_capacities = create_knapsack_capacities_integer(3, 50, 75)
print("Knapsack Capacities:", knapsack_capacities)


       item  value  cost
0    Item_1     23    27
1    Item_2     99    37
2    Item_3     97    33
3    Item_4     75    10
4    Item_5     27    18
5    Item_6     47    17
6    Item_7     53    21
7    Item_8     43    18
8    Item_9     93     8
9   Item_10     91    11
10  Item_11     52    15
11  Item_12     53    21
12  Item_13     31    41
13  Item_14     65    32
14  Item_15     62    26
15  Item_16     31    14
16  Item_17     36    23
17  Item_18     77     8
18  Item_19     56    11
19  Item_20     92    18
20  Item_21     35    46
21  Item_22     40    12
22  Item_23     45    47
23  Item_24     38    19
24  Item_25     73    10
25  Item_26     79    50
26  Item_27     97    31
27  Item_28     66    12
28  Item_29     89    50
29  Item_30     61    44
30  Item_31     68    44
31  Item_32     18    30
32  Item_33     25    21
33  Item_34     46    50
34  Item_35     76    21
35  Item_36     68    25
36  Item_37     74     6
37  Item_38     99    45
38  Item_39     26    42


In [67]:
def multiple_knapsack(values, costs, capacities):
    """
    Solve the multiple knapsack problem to maximize total value while respecting capacity constraints
    and ensuring each item is assigned to at most one knapsack.

    Parameters:
    - values (List[int]): List of item values.
    - costs (List[int]): List of item costs (weights).
    - capacities (List[int]): List of knapsack capacities.

    Returns:
    - List[List[int]]: The items assigned to each knapsack.
    - int: The maximum total value achieved.
    """
    num_items = len(values)
    num_knapsacks = len(capacities)

    # DP table: dp[i][j] represents the max value for the first i items and total capacity j
    total_capacity = sum(capacities)
    dp = [[0] * (total_capacity + 1) for _ in range(num_items + 1)]

    # Backtracking table to track which items are placed in which knapsack
    item_assignment = [[[] for _ in range(total_capacity + 1)] for _ in range(num_items + 1)]

    for i in range(1, num_items + 1):
        for cap in range(total_capacity + 1):
            # Case 1: Skip the current item
            dp[i][cap] = dp[i - 1][cap]
            item_assignment[i][cap] = item_assignment[i - 1][cap][:]

            # Case 2: Use the current item if it fits in the capacity
            if costs[i - 1] <= cap:
                new_value = dp[i - 1][cap - costs[i - 1]] + values[i - 1]
                if new_value > dp[i][cap]:
                    dp[i][cap] = new_value
                    item_assignment[i][cap] = item_assignment[i - 1][cap - costs[i - 1]] + [(i - 1)]

    # Assign items to knapsacks
    knapsack_items = [[] for _ in range(num_knapsacks)]
    remaining_capacity = capacities[:]
    used_items = set()

    for i in range(num_items, 0, -1):
        for k in range(num_knapsacks):
            if item_assignment[i][sum(remaining_capacity)] and item_assignment[i][sum(remaining_capacity)][-1] not in used_items:
                item = item_assignment[i][sum(remaining_capacity)].pop()
                if costs[item] <= remaining_capacity[k]:
                    knapsack_items[k].append(item)
                    used_items.add(item)
                    remaining_capacity[k] -= costs[item]

    total_value = dp[num_items][sum(capacities)]
    return knapsack_items, total_value

values = df['value'].astype(int).tolist()
costs = df['cost'].astype(int).tolist()
capacities = knapsack_capacities

knapsack_solution, max_value = multiple_knapsack(values, costs, capacities)
knapsack_solution, max_value

([[42, 39, 36, 21, 8], [27, 24, 19, 17, 9], [34, 18]], 1104)

In [68]:
def check_solution_correctness(items_df, knapsack_solution):
    """
    Check if the sum of the values of items in the knapsack solution matches the reported total value.

    Parameters:
    - items_df (DataFrame): A pandas DataFrame with columns 'value' and 'cost'.
    - knapsack_solution (List[List[int]]): The items assigned to each knapsack.

    Returns:
    - int: The computed total value from the solution.
    """
    total_value = 0
    for knapsack in knapsack_solution:
        total_value += items_df.loc[knapsack, 'value'].sum()
    return total_value


# Example usage
items_df = df
capacities = knapsack_capacities
knapsack_solution = knapsack_solution
reported_value = max_value

# Verify the solution
computed_value = check_solution_correctness(items_df, knapsack_solution)
is_correct = computed_value == reported_value

computed_value, is_correct


(920, False)

In [69]:
knapsack_capacities

[67, 63, 72]

In [70]:
sum(knapsack_capacities)

202

In [71]:
def multiple_knapsack(values, costs, capacities):
    """
    Solve the multiple knapsack problem to maximize total value while respecting capacity constraints
    and ensuring each item is assigned to at most one knapsack.

    Parameters:
    - values (List[int]): List of item values.
    - costs (List[int]): List of item costs (weights).
    - capacities (List[int]): List of knapsack capacities.

    Returns:
    - List[List[int]]: The items assigned to each knapsack.
    - int: The maximum total value achieved.
    """
    num_items = len(values)
    num_knapsacks = len(capacities)

    # DP table: dp[i][k][cap] - max value using first i items in knapsack k with capacity cap
    dp = [[[0 for _ in range(cap + 1)] for cap in capacities] for _ in range(num_items + 1)]

    # Backtracking structure to track assignments
    assignments = [[[] for _ in range(cap + 1)] for cap in capacities]

    for i in range(1, num_items + 1):
        for k in range(num_knapsacks):
            for cap in range(capacities[k] + 1):
                # Case 1: Skip the item
                dp[i][k][cap] = dp[i - 1][k][cap]

                # Case 2: Place the item in knapsack k (if it fits)
                if costs[i - 1] <= cap:
                    value_with_item = dp[i - 1][k][cap - costs[i - 1]] + values[i - 1]
                    if value_with_item > dp[i][k][cap]:
                        dp[i][k][cap] = value_with_item
                        assignments[k][cap] = assignments[k][cap - costs[i - 1]] + [i - 1]

    # Collect the assignments
    knapsack_items = [[] for _ in range(num_knapsacks)]
    total_value = 0

    for k in range(num_knapsacks):
        remaining_capacity = capacities[k]
        for i in range(num_items, 0, -1):
            if dp[i][k][remaining_capacity] != dp[i - 1][k][remaining_capacity]:
                item = assignments[k][remaining_capacity].pop()
                knapsack_items[k].append(item)
                remaining_capacity -= costs[item]
                total_value += values[item]

    return knapsack_items, total_value


# Example usage
values = [60, 100, 120, 80, 90]
costs = [10, 20, 30, 15, 25]
capacities = [50, 50, 50]  # Three knapsacks with capacities 50 each

knapsack_solution, max_value = multiple_knapsack(values, costs, capacities)
print("Knapsack Solution:", knapsack_solution)
print("Maximum Value:", max_value)


Knapsack Solution: [[3, 3, 1], [3, 3, 1], [3, 3, 1]]
Maximum Value: 780


In [72]:
from ortools.linear_solver import pywraplp

def multiple_knapsack(values, weights, capacities):
    """
    Solve the multiple knapsack problem using OR-Tools.

    Parameters:
    - values (List[int]): List of item values.
    - weights (List[int]): List of item weights.
    - capacities (List[int]): List of knapsack capacities.

    Returns:
    - List[List[int]]: The items assigned to each knapsack.
    - int: The maximum total value achieved.
    """
    num_items = len(values)
    num_knapsacks = len(capacities)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return None, None

    # Decision variables: x[i][j] is 1 if item i is placed in knapsack j
    x = {}
    for i in range(num_items):
        for j in range(num_knapsacks):
            x[i, j] = solver.BoolVar(f"x_{i}_{j}")

    # Objective: Maximize total value
    solver.Maximize(solver.Sum(values[i] * x[i, j] for i in range(num_items) for j in range(num_knapsacks)))

    # Constraints: Each item can be placed in at most one knapsack
    for i in range(num_items):
        solver.Add(solver.Sum(x[i, j] for j in range(num_knapsacks)) <= 1)

    # Constraints: Knapsack capacities
    for j in range(num_knapsacks):
        solver.Add(solver.Sum(weights[i] * x[i, j] for i in range(num_items)) <= capacities[j])

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Retrieve the solution
        knapsack_items = [[] for _ in range(num_knapsacks)]
        total_value = solver.Objective().Value()
        for i in range(num_items):
            for j in range(num_knapsacks):
                if x[i, j].solution_value() > 0:
                    knapsack_items[j].append(i)
        return knapsack_items, total_value
    else:
        return None, None


# Example usage
#values = [60, 100, 120, 80, 90, 10, 20, 30, 10, 20, 50, 20]
#weights = [10, 20, 30, 15, 25, 10, 40, 20, 50, 20, 40, 10]
#capacities = [50, 50, 50]  # Three knapsacks

values = df['value'].astype(int).tolist()
costs = df['cost'].astype(int).tolist()
capacities = knapsack_capacities

knapsack_solution, max_value = multiple_knapsack(values, costs, capacities)
print("Knapsack Solution:", knapsack_solution)
print("Maximum Value:", max_value)


Knapsack Solution: [[10, 26, 34], [3, 27, 39, 42], [8, 9, 17, 18, 19, 24, 36]]
Maximum Value: 1104.0000000000002


In [73]:
sum(values)

3033

In [74]:
capacities

[67, 63, 72]

In [75]:
def check_solution_correctness(items_df, knapsack_solution):
    """
    Check if the sum of the values of items in the knapsack solution matches the reported total value.

    Parameters:
    - items_df (DataFrame): A pandas DataFrame with columns 'value' and 'cost'.
    - knapsack_solution (List[List[int]]): The items assigned to each knapsack.

    Returns:
    - int: The computed total value from the solution.
    """
    total_value = 0
    for knapsack in knapsack_solution:
        total_value += items_df.loc[knapsack, 'value'].sum()
    return total_value


# Example usage
items_df = df
capacities = knapsack_capacities
knapsack_solution = knapsack_solution
reported_value = round(max_value)

# Verify the solution
computed_value = check_solution_correctness(items_df, knapsack_solution)
is_correct = computed_value == reported_value

computed_value, is_correct


(1104, True)

In [76]:
import pandas as pd

def check_solution_correctness_and_capacity(items_df, knapsack_solution, capacities):
    """
    Check if the sum of the values of items in the knapsack solution matches the reported total value,
    and compute the capacity used in each knapsack.

    Parameters:
    - items_df (DataFrame): A pandas DataFrame with columns 'value' and 'cost'.
    - knapsack_solution (List[List[int]]): The items assigned to each knapsack.
    - capacities (List[int]): List of knapsack capacities.

    Returns:
    - int: The computed total value from the solution.
    - List[int]: The used capacity for each knapsack.
    """
    total_value = 0
    used_capacities = []

    for i, knapsack in enumerate(knapsack_solution):
        knapsack_value = items_df.loc[knapsack, 'value'].sum()
        knapsack_cost = items_df.loc[knapsack, 'cost'].sum()
        total_value += knapsack_value
        used_capacities.append(knapsack_cost)

    return total_value, used_capacities

# Example usage
items_df = df
capacities = knapsack_capacities
knapsack_solution = knapsack_solution
reported_value = round(max_value)

# Verify the solution
computed_value, used_capacities = check_solution_correctness_and_capacity(df, knapsack_solution, knapsack_capacities)
is_correct = computed_value == round(max_value)

df, computed_value, used_capacities, is_correct


(       item  value  cost
 0    Item_1     23    27
 1    Item_2     99    37
 2    Item_3     97    33
 3    Item_4     75    10
 4    Item_5     27    18
 5    Item_6     47    17
 6    Item_7     53    21
 7    Item_8     43    18
 8    Item_9     93     8
 9   Item_10     91    11
 10  Item_11     52    15
 11  Item_12     53    21
 12  Item_13     31    41
 13  Item_14     65    32
 14  Item_15     62    26
 15  Item_16     31    14
 16  Item_17     36    23
 17  Item_18     77     8
 18  Item_19     56    11
 19  Item_20     92    18
 20  Item_21     35    46
 21  Item_22     40    12
 22  Item_23     45    47
 23  Item_24     38    19
 24  Item_25     73    10
 25  Item_26     79    50
 26  Item_27     97    31
 27  Item_28     66    12
 28  Item_29     89    50
 29  Item_30     61    44
 30  Item_31     68    44
 31  Item_32     18    30
 32  Item_33     25    21
 33  Item_34     46    50
 34  Item_35     76    21
 35  Item_36     68    25
 36  Item_37     74     6
 37  Item_38

In [77]:
def multiple_knapsack_2bags_corrected(values, weights, capacities):
    """
    Solve the double knapsack problem using Dynamic Programming,
    ensuring that each item is placed in at most one knapsack.

    Parameters:
    - values (List[int]): List of item values.
    - weights (List[int]): List of item weights.
    - capacities (List[int]): List of knapsack capacities (2 knapsacks).

    Returns:
    - List[List[int]]: Items assigned to each knapsack.
    - int: Maximum total value achieved.
    """
    num_items = len(values)
    capacity_1, capacity_2 = capacities

    # 3D DP table: dp[i][c1][c2]
    dp = [[[0 for _ in range(capacity_2 + 1)] for _ in range(capacity_1 + 1)] for _ in range(num_items + 1)]

    # Fill the DP table
    for i in range(1, num_items + 1):
        for c1 in range(capacity_1 + 1):
            for c2 in range(capacity_2 + 1):
                # Case 1: Skip the item
                dp[i][c1][c2] = dp[i - 1][c1][c2]

                # Case 2: Place the item in knapsack 1 (if it fits)
                if weights[i - 1] <= c1:
                    dp[i][c1][c2] = max(dp[i][c1][c2], dp[i - 1][c1 - weights[i - 1]][c2] + values[i - 1])

                # Case 3: Place the item in knapsack 2 (if it fits)
                if weights[i - 1] <= c2:
                    dp[i][c1][c2] = max(dp[i][c1][c2], dp[i - 1][c1][c2 - weights[i - 1]] + values[i - 1])

    # Backtrack to find the item assignments
    knapsack_1_items = []
    knapsack_2_items = []
    c1, c2 = capacity_1, capacity_2

    for i in range(num_items, 0, -1):
        if dp[i][c1][c2] == dp[i - 1][c1][c2]:
            # Item i-1 was not included in either knapsack
            continue
        if c1 >= weights[i - 1] and dp[i][c1][c2] == dp[i - 1][c1 - weights[i - 1]][c2] + values[i - 1]:
            # Item i-1 was placed in knapsack 1
            knapsack_1_items.append(i - 1)
            c1 -= weights[i - 1]
        elif c2 >= weights[i - 1] and dp[i][c1][c2] == dp[i - 1][c1][c2 - weights[i - 1]] + values[i - 1]:
            # Item i-1 was placed in knapsack 2
            knapsack_2_items.append(i - 1)
            c2 -= weights[i - 1]

    total_value = dp[num_items][capacity_1][capacity_2]

    return [knapsack_1_items, knapsack_2_items], total_value


#knapsack_solution, max_value = multiple_knapsack_2bags_corrected(values, costs, capacities)
#print("Knapsack Solution:", knapsack_solution)
#print("Maximum Value:", max_value)


In [78]:
def multiple_knapsack_nbags(values, weights, capacities):
    """
    Solve the multiple knapsack problem with n knapsacks using Dynamic Programming.

    Parameters:
    - values (List[int]): List of item values.
    - weights (List[int]): List of item weights.
    - capacities (List[int]): List of knapsack capacities.

    Returns:
    - List[List[int]]: Items assigned to each knapsack.
    - int: Maximum total value achieved.
    """
    num_items = len(values)
    num_knapsacks = len(capacities)

    # Initialize DP table: dp[i][c1][c2]...[cm]
    dp = {}

    # Recursive helper function
    def solve(i, capacities_tuple):
        if i == 0:
            return 0
        if (i, capacities_tuple) in dp:
            return dp[(i, capacities_tuple)]

        # Case 1: Skip the current item
        max_value = solve(i - 1, capacities_tuple)

        # Case 2: Try placing the current item in each knapsack
        for k in range(num_knapsacks):
            if capacities_tuple[k] >= weights[i - 1]:
                new_capacities = list(capacities_tuple)
                new_capacities[k] -= weights[i - 1]
                max_value = max(max_value, solve(i - 1, tuple(new_capacities)) + values[i - 1])

        dp[(i, capacities_tuple)] = max_value
        return max_value

    # Compute the maximum value
    max_value = solve(num_items, tuple(capacities))

    # Backtrack to find the item assignments
    knapsack_items = [[] for _ in range(num_knapsacks)]
    capacities_left = list(capacities)

    for i in range(num_items, 0, -1):
        was_placed = False
        for k in range(num_knapsacks):
            if capacities_left[k] >= weights[i - 1]:
                new_capacities = list(capacities_left)
                new_capacities[k] -= weights[i - 1]
                if dp.get((i - 1, tuple(new_capacities)), 0) + values[i - 1] == dp.get((i, tuple(capacities_left)), 0):
                    knapsack_items[k].append(i - 1)
                    capacities_left[k] -= weights[i - 1]
                    was_placed = True
                    break
        if not was_placed:
            continue

    return knapsack_items, max_value


#knapsack_solution, max_value = multiple_knapsack_nbags(values, costs, capacities)
#print("Knapsack Solution:", knapsack_solution)
#print("Maximum Value:", max_value)


Knapsack Solution: [[42, 39, 36, 27, 17], [34, 26, 18], [24, 19, 10, 9, 8, 3]]
Maximum Value: 1104


In [79]:
# Add tempral dependency

In [80]:
import pandas as pd
import numpy as np

def create_items_dataframe_with_values(num_items, num_knapsacks, value_min, value_max, cost_min, cost_max):
    """
    Create a pandas DataFrame with N items, each having a list of randomly sampled integer values
    (one for each knapsack) and a cost.

    Parameters:
    - num_items (int): Number of items to generate.
    - num_knapsacks (int): Number of knapsacks.
    - value_min (int): Minimum possible value for each 'value' in the list.
    - value_max (int): Maximum possible value for each 'value' in the list.
    - cost_min (int): Minimum possible cost for the 'cost' column.
    - cost_max (int): Maximum possible cost for the 'cost' column.

    Returns:
    - DataFrame: A pandas DataFrame with columns ['item', 'value', 'cost'].
    """
    data = {
        'item': [f'Item_{i+1}' for i in range(num_items)],
        'value': [list(np.random.randint(value_min, value_max + 1, num_knapsacks)) for _ in range(num_items)],
        'cost': np.random.randint(cost_min, cost_max + 1, num_items),
    }
    return pd.DataFrame(data)

# Example usage
num_items = 10
num_knapsacks = 3
value_min = 50
value_max = 150
cost_min = 5
cost_max = 30

df = create_items_dataframe_with_values(num_items, num_knapsacks, value_min, value_max, cost_min, cost_max)
knapsack_capacities = create_knapsack_capacities_integer(num_knapsacks, 50, 75)


In [81]:
from ortools.linear_solver import pywraplp

def multiple_knapsack_with_dependent_values(values, weights, capacities):
    """
    Solve the multiple knapsack problem with knapsack-dependent item values using OR-Tools.

    Parameters:
    - values (List[List[int]]): 2D list where values[i][j] is the value of item i in knapsack j.
    - weights (List[int]): List of item weights.
    - capacities (List[int]): List of knapsack capacities.

    Returns:
    - List[List[int]]: The items assigned to each knapsack.
    - int: The maximum total value achieved.
    """
    num_items = len(weights)
    num_knapsacks = len(capacities)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return None, None

    # Decision variables: x[i][j] is 1 if item i is placed in knapsack j
    x = {}
    for i in range(num_items):
        for j in range(num_knapsacks):
            x[i, j] = solver.BoolVar(f"x_{i}_{j}")

    # Objective: Maximize total value
    solver.Maximize(
        solver.Sum(values[i][j] * x[i, j] for i in range(num_items) for j in range(num_knapsacks))
    )

    # Constraints: Each item can be placed in at most one knapsack
    for i in range(num_items):
        solver.Add(solver.Sum(x[i, j] for j in range(num_knapsacks)) <= 1)

    # Constraints: Knapsack capacities
    for j in range(num_knapsacks):
        solver.Add(solver.Sum(weights[i] * x[i, j] for i in range(num_items)) <= capacities[j])

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Retrieve the solution
        knapsack_items = [[] for _ in range(num_knapsacks)]
        total_value = solver.Objective().Value()
        for i in range(num_items):
            for j in range(num_knapsacks):
                if x[i, j].solution_value() > 0:
                    knapsack_items[j].append(i)
        return knapsack_items, total_value
    else:
        return None, None


# Example usage
# Values depend on the knapsack:
#values = [
#    [60, 70, 80],  # Item 0: value depends on the knapsack
#    [100, 90, 120],  # Item 1
#    [120, 110, 100],  # Item 2
#    [80, 85, 75],  # Item 3
#    [90, 95, 85]  # Item 4
#]
#weights = [10, 20, 30, 15, 25]
#capacities = [50, 50, 50]  # Three knapsacks

values = df['value'].tolist()
costs = df['cost'].astype(int).tolist()
capacities = knapsack_capacities

# Solve the problem
knapsack_solution, max_value = multiple_knapsack_with_dependent_values(values, costs, capacities)
print("Knapsack Solution:", knapsack_solution)
print("Maximum Value:", max_value)


Knapsack Solution: [[0, 6], [2, 4, 9], [1, 3, 7]]
Maximum Value: 984.0


In [82]:
import pandas as pd
import numpy as np

def create_items_dataframe_with_values_and_due_dates(num_items, num_knapsacks, value_min, value_max, cost_min, cost_max, due_date_min, due_date_max):
    """
    Create a pandas DataFrame with N items, each having a list of randomly sampled integer values
    (one for each knapsack), a cost, and a due date.

    Parameters:
    - num_items (int): Number of items to generate.
    - num_knapsacks (int): Number of knapsacks.
    - value_min (int): Minimum possible value for each 'value' in the list.
    - value_max (int): Maximum possible value for each 'value' in the list.
    - cost_min (int): Minimum possible cost for the 'cost' column.
    - cost_max (int): Maximum possible cost for the 'cost' column.
    - due_date_min (int): Minimum possible due date (knapsack index).
    - due_date_max (int): Maximum possible due date (knapsack index).

    Returns:
    - DataFrame: A pandas DataFrame with columns ['item', 'value', 'cost', 'due_date'].
    """
    data = {
        'item': [f'Item_{i+1}' for i in range(num_items)],
        'value': [[1]*num_knapsacks]*num_items,#[list(np.random.randint(value_min, value_max + 1, num_knapsacks)) for _ in range(num_items)],
        'cost': np.random.randint(cost_min, cost_max + 1, num_items),
        'due_date': np.random.randint(due_date_min, due_date_max + 1, num_items)
    }
    return pd.DataFrame(data)

# Example usage
num_items = 10
num_knapsacks = 3
value_min = 50
value_max = 150
cost_min = 5
cost_max = 30
due_date_min = 1
due_date_max = num_knapsacks

df = create_items_dataframe_with_values_and_due_dates(
    num_items, num_knapsacks, value_min, value_max, cost_min, cost_max, due_date_min, due_date_max
)

df


,item,value,cost,due_date
0,Item_1,"[1, 1, 1]",16,2
1,Item_2,"[1, 1, 1]",14,3
2,Item_3,"[1, 1, 1]",20,3
3,Item_4,"[1, 1, 1]",21,2
4,Item_5,"[1, 1, 1]",27,1
5,Item_6,"[1, 1, 1]",25,3
6,Item_7,"[1, 1, 1]",11,1
7,Item_8,"[1, 1, 1]",5,2
8,Item_9,"[1, 1, 1]",27,2
9,Item_10,"[1, 1, 1]",27,1


In [83]:
from ortools.linear_solver import pywraplp

def multiple_knapsack_with_penalties(values, weights, capacities, due_dates, penalty_per_day):
    """
    Solve the multiple knapsack problem with penalties for late completion using OR-Tools.

    Parameters:
    - values (List[List[int]]): 2D list where values[i][j] is the value of job i on day j.
    - weights (List[int]): List of job weights.
    - capacities (List[int]): List of daily capacities.
    - due_dates (List[int]): List of due dates for each job.
    - penalty_per_day (int): Penalty per day of lateness.

    Returns:
    - List[List[int]]: The jobs assigned to each day.
    - int: The total objective value (value - penalties).
    """
    num_jobs = len(weights)
    num_days = len(capacities)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return None, None

    # Decision variables: x[i][j] is 1 if job i is assigned to day j
    x = {}
    for i in range(num_jobs):
        for j in range(num_days):
            x[i, j] = solver.BoolVar(f"x_{i}_{j}")

    # Penalty variables: p[i] is the penalty for job i
    penalties = [solver.NumVar(0, solver.infinity(), f"p_{i}") for i in range(num_jobs)]

    # Objective: Maximize total value minus penalties
    solver.Maximize(
        solver.Sum(values[i][j] * x[i, j] for i in range(num_jobs) for j in range(num_days))
        - solver.Sum(penalties[i] for i in range(num_jobs))
    )

    # Constraints: Each job can be assigned to at most one day
    for i in range(num_jobs):
        solver.Add(solver.Sum(x[i, j] for j in range(num_days)) <= 1)

    # Constraints: Daily capacities
    for j in range(num_days):
        solver.Add(solver.Sum(weights[i] * x[i, j] for i in range(num_jobs)) <= capacities[j])

    # Constraints: Define penalties
    for i in range(num_jobs):
        due_date = due_dates[i]
        solver.Add(penalties[i] >= solver.Sum((j - due_date) * penalty_per_day * x[i, j] for j in range(num_days)))

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Retrieve the solution
        schedule = [[] for _ in range(num_days)]
        total_value = solver.Objective().Value()
        for i in range(num_jobs):
            for j in range(num_days):
                if x[i, j].solution_value() > 0:
                    schedule[j].append(i)
        return schedule, total_value
    else:
        return None, None


# Example usage
#values = [
#    [100, 90, 80],  # Job 0: values for day 1, 2, 3
#    [120, 110, 100],  # Job 1
#    [140, 130, 120],  # Job 2
#]
#weights = [10, 15, 20]
#capacities = [30, 30, 30]
#due_dates = [1, 2, 2]  # Job 0 due on day 1, Job 1 and 2 due on day 2
#penalty_per_day = 10

values = df['value'].tolist()
costs = df['cost'].astype(int).tolist()
capacities = knapsack_capacities
due_dates = df['due_date'].tolist()
penalty_per_day = 10

# Solve the problem
schedule, total_value = multiple_knapsack_with_penalties(values, costs, capacities, due_dates, penalty_per_day)
print("Job Schedule:", schedule)
print("Total Objective Value:", total_value)


Job Schedule: [[1, 4, 7], [0, 2, 6], [3, 5, 8]]
Total Objective Value: 9.0


In [84]:
from ortools.linear_solver import pywraplp

def multiple_knapsack_with_penalties(values, weights, capacities, due_dates, penalty_per_day):
    """
    Solve the multiple knapsack problem with penalties for late completion using OR-Tools.

    Parameters:
    - values (List[List[int]]): 2D list where values[i][j] is the value of job i on day j.
    - weights (List[int]): List of job weights.
    - capacities (List[int]): List of daily capacities.
    - due_dates (List[int]): List of due dates for each job.
    - penalty_per_day (int): Penalty per day of lateness.

    Returns:
    - List[List[int]]: The jobs assigned to each day.
    - int: The total objective value (value - penalties).
    """
    num_jobs = len(weights)
    num_days = len(capacities)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return None, None

    # Decision variables: x[i][j] is 1 if job i is assigned to day j
    x = {}
    for i in range(num_jobs):
        for j in range(num_days):
            x[i, j] = solver.BoolVar(f"x_{i}_{j}")

    # Penalty variables: p[i] is the penalty for job i
    penalties = [solver.NumVar(0, solver.infinity(), f"p_{i}") for i in range(num_jobs)]

    # Objective: Maximize total value minus penalties
    solver.Maximize(
        solver.Sum(values[i][j] * x[i, j] for i in range(num_jobs) for j in range(num_days))
        - solver.Sum(penalties[i] for i in range(num_jobs))
    )

    # Constraints: Each job can be assigned to at most one day
    for i in range(num_jobs):
        solver.Add(solver.Sum(x[i, j] for j in range(num_days)) <= 1)

    # Constraints: Daily capacities
    for j in range(num_days):
        solver.Add(solver.Sum(weights[i] * x[i, j] for i in range(num_jobs)) <= capacities[j])

    # Constraints: Define penalties only for jobs scheduled after their due date
    for i in range(num_jobs):
        due_date = due_dates[i] - 1  # Convert to 0-based index
        solver.Add(penalties[i] >= solver.Sum(
            penalty_per_day * (j - due_date) * x[i, j]
            for j in range(due_date + 1, num_days)
        ))

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Retrieve the solution
        schedule = [[] for _ in range(num_days)]
        total_value = solver.Objective().Value()
        for i in range(num_jobs):
            for j in range(num_days):
                if x[i, j].solution_value() > 0:
                    schedule[j].append(i)
        return schedule, total_value
    else:
        return None, None


# Example usage
values = df['value'].tolist()
costs = df['cost'].astype(int).tolist()
capacities = knapsack_capacities
due_dates = df['due_date'].tolist()
penalty_per_day = 10

# Solve the problem
schedule, total_value = multiple_knapsack_with_penalties(values, costs, capacities, due_dates, penalty_per_day)
print("Job Schedule:", schedule)
print("Total Objective Value:", total_value)


Job Schedule: [[0, 4, 7], [3, 8], [1, 2, 5]]
Total Objective Value: 8.0


In [99]:
from ortools.linear_solver import pywraplp

def multiple_knapsack_with_penalties(values, weights, capacities, due_dates, penalty_per_day):
    """
    Solve the multiple knapsack problem with penalties for late completion using OR-Tools.

    Parameters:
    - values (List[List[int]]): 2D list where values[i][j] is the value of job i on day j.
    - weights (List[int]): List of job weights.
    - capacities (List[int]): List of daily capacities.
    - due_dates (List[int]): List of due dates for each job.
    - penalty_per_day (int): Penalty per day of lateness.

    Returns:
    - List[List[int]]: The jobs assigned to each day.
    - int: The total objective value (value - penalties).
    """
    num_jobs = len(weights)
    num_days = len(capacities)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver("SCIP")
    if not solver:
        return None, None

    # Decision variables: x[i][j] is 1 if job i is assigned to day j
    x = {}
    for i in range(num_jobs):
        for j in range(num_days):
            x[i, j] = solver.BoolVar(f"x_{i}_{j}")

    # Objective: Maximize total value minus penalties
    objective = solver.Sum(values[i][j] * x[i, j] for i in range(num_jobs) for j in range(num_days))

    # Add penalties for jobs scheduled after their due date
    for i in range(num_jobs):
        due_date = due_dates[i] - 1  # Convert to 0-based index
        for j in range(due_date + 1, num_days):
            objective -= penalty_per_day * (j - due_date) * x[i, j]

    # Add penalties for missed jobs
    for i in range(num_jobs):
        missed_penalty = penalty_per_day * (num_days + 1 - due_dates[i])
        objective -= missed_penalty * (1 - solver.Sum(x[i, j] for j in range(num_days)))

    solver.Maximize(objective)

    # Constraints: Each job can be assigned to at most one day
    for i in range(num_jobs):
        solver.Add(solver.Sum(x[i, j] for j in range(num_days)) <= 1)

    # Constraints: Daily capacities
    for j in range(num_days):
        solver.Add(solver.Sum(weights[i] * x[i, j] for i in range(num_jobs)) <= capacities[j])

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        # Retrieve the solution
        schedule = [[] for _ in range(num_days)]
        total_value = solver.Objective().Value()
        for i in range(num_jobs):
            for j in range(num_days):
                if x[i, j].solution_value() > 0:
                    schedule[j].append(i)
        return schedule, total_value
    else:
        return None, None


# Example usage
values = [
    [1, 1, 1],  # Job 0
    [1, 1, 1],  # Job 1
    [1, 1, 1],  # Job 2
    [1, 1, 1],  # Job 3
    [1, 1, 1],  # Job 4
    [1, 1, 1],  # Job 5
    [1, 1, 1],  # Job 6
    [1, 1, 1],  # Job 7
    [1, 1, 1],  # Job 8
    [1, 1, 1],  # Job 9
]
weights = [1]*10#[16, 14, 20, 21, 27, 25, 11, 5, 27, 27]
capacities = [2, 2, 2]
due_dates = [2, 3, 3, 2, 1, 3, 1, 2, 2, 1]
penalty_per_day = 10

# Solve the problem
schedule, total_value = multiple_knapsack_with_penalties(values, weights, capacities, due_dates, penalty_per_day)
print("Job Schedule:", schedule)
print("Total Objective Value:", total_value)


Job Schedule: [[4, 6], [7, 9], [0, 2]]
Total Objective Value: -74.0
